In [3]:
# Types of modules
import sys, os, math, django
print(sys)  # built-in
print(os)  # Python files
print(math)  # C lib
print(django)  # Third party

<module 'sys' (built-in)>
<module 'os' from '/usr/lib/python3.8/os.py'>
<module 'math' (built-in)>
<module 'django' from '/home/matan/.local/lib/python3.8/site-packages/django/__init__.py'>


In [7]:
# Import system - done with dicts :)

code = '''
def hello():
    print('Hello, world!')
'''
import types
m = types.ModuleType('m')
print(m)
print(m.__dict__)
exec(code, m.__dict__)
print('hello' in m.__dict__)
m.hello()

<module 'm'>
{'__name__': 'm', '__doc__': None, '__package__': None, '__loader__': None, '__spec__': None}
True
Hello, world!


In [14]:
# Implementing test fixtures
import inspect

def f():
    print(1)

def g(x):
    print(x)

def h(x, y):
    print(x + y)

def call(f, **available):
    # Retrieve the available arguments that `f` expects to receive.
    expected = inspect.getfullargspec(f).args
    kwargs = {k: v for k, v in available.items() if k in expected}
    return f(**kwargs)

call(f, x=2, y=3)
call(g, x=2, y=3)
call(h, x=2, y=3)

1
2
5


In [1]:
# Abstract factory in Python is much simpler due to duck typing (no need 
# for the abstract class) and classes being objects (no need for factory
# functions, just hold the concrete class as a class field of the factory).
class ProductA1:
    def use(self):
        print('A1')
class ProductB1:
    def use(self):
        print('B1')
class Factory1:
    ProductA = ProductA1
    ProductB = ProductB1

class ProductA2:
    def use(self):
        print('A2')
class ProductB2:
    def use(self):
        print('B2')
class Factory2:
    ProductA = ProductA2
    ProductB = ProductB2

def create_and_use_a(factory):
    a = factory.ProductA()
    a.use()
create_and_use_a(Factory1)
create_and_use_a(Factory2)

A1
A2


In [6]:
# Singleton - can always get around this in python.

class Singleton:
    instance = None

    def __new__(cls):
        if cls.instance is None:
            cls.instance = super().__new__(cls)
        return cls.instance

i1 = Singleton()
i2 = Singleton()
i3 = object.__new__(Singleton)
print(i1 is i2, i1 is i3)

True False


In [11]:
# Borg - a cool take on singleton, where each object is different, but
# their state is shared.
class Borg:
    _dict = {'x': 1}
    def __init__(self):
        self.__dict__ = Borg._dict

b1 = Borg()
b2 = Borg()
print(b1 is b2)
b2.x = 2
print(b1.x)

False
2


In [23]:
# Factory method
import math

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    @staticmethod
    def from_cartesian(x, y):
        return Point(x, y)
    
    @staticmethod
    def from_polar(r, t):
        return Point(r * math.cos(t), r * math.sin(t))

p1 = Point.from_cartesian(0, 1)
p2 = Point.from_polar(1, math.pi)
print(p1.x, p1.y, p2.x, p2.y)


0 1 0.0007963267107332633 0.9999996829318346


In [16]:
# Trace Trick
import sys

class Tracer:
    def __init__(self):
        sys.settrace(self.tracer)

    def tracer(self, frame, event, arg):
        sys.settrace(None)
        self.order = frame.f_code.co_names[3:]
    
    def __call__(self, cls):
        cls._order = self.order
        return cls

@Tracer()
class A:
    x = 1
    y = 2

# The fields of A exist before we actually create A; therefore they are 
# in the frame for tracer.
A._order

('x', 'y')

In [18]:
# Selfless functions
# Decorate all methods of the class, to replace: LOAD_GLOBAL [self]
# with LOAD_FAST 0. We then need to replace LOAD_FAST n with
# LOAD_FAST n+1. Done by changing f.__code__.co_code via
# types.CodeType.
#
# Another method is using `compile` with types.FunctionType.
import inspect, types

class Proxy(dict):
    def __init__(self, globals):
        self.globals = globals
        self.self = None

    def __getitem__(self, key):
        if key == 'self':
            return self.self
        return self.globals[key]

def patch(method):
    proxy = Proxy(method.__globals__)
    f = types.FunctionType(method.__code__, proxy)

    def wrapper(self, *args, **kwargs):
        proxy.self = self
        return f(*args, **kwargs)
    
    return wrapper

def selfless(cls):
    for k, v in cls.__dict__.items():
        if inspect.isfunction(v):
            setattr(cls, k, patch(v))
    return cls

@selfless
class A:
    def __init__(x):
        self.x = x
    def f():
        return self.x

a = A(1)
a.f()

NameError: name 'types' is not defined